# Librerias

In [1]:
%%capture 
!pip install transformers

In [2]:
from sklearn.model_selection import train_test_split
import torch

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
%%capture
!pip install wandb
import wandb
wandb.login()
# api key 64d2528e8d76865170c16bf804e7172bb3780c79

wandb: Paste an API key from your profile and hit enter: ··········


In [5]:
wandb.login()

wandb: Currently logged in as: lucialarraona (use `wandb login --relogin` to force relogin)


True

# Carga de datos
- incluidas nuevas columnas con concatenacion de review_body y categoria, review_body y titulo
- nuevos indices para star rating, de 0-4 en vez de 1-5
- numericalizar las categorias de los productos, 31 en total

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load data
import pandas as pd
path1 = "drive/MyDrive/TFG_1/Datasets_procesados/train/amazonEN_train.csv"
path2 = "drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonEN_valid.csv"
path3 = "drive/MyDrive/TFG_1/Datasets_procesados/test/amazonEN_test.csv"
df_train = pd.read_csv(path1)
df_train = df_train.head(100000)
df_valid = pd.read_csv(path2)
df_valid = df_valid
df_test = pd.read_csv(path3)
df_test = df_test

In [ ]:
df_train.head(1)

,review_id,product_id,reviewer_id,star_rating,review_body,review_title,language,product_category,text_title,text_title_cat,text_cat,product_category_cod
0,en_0953486,product_en_0381770,reviewer_en_0978532,4,"Great, great batteries. Arrived on time and pa...",Strip of 10 Fresh Maxell LR1130 Batteries,en,electronics,Strip of 10 Fresh Maxell LR1130 Batteries Grea...,Strip of 10 Fresh Maxell LR1130 Batteries Grea...,"Great, great batteries. Arrived on time and pa...",9


# Carga modelo BERT // distilBERT
- incluye tokenizer

In [ ]:
# Bert 

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
from transformers import BertModel, BertConfig, BertTokenizer
from transformers import BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
# DistilBert

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


# BertForSequenceClassification - RATING
- Clasificación Rating a partir de review_body

In [ ]:
%env WANDB_PROJECT = RATING CLASSIFICATION distilBERT

env: WANDB_PROJECT=RATING CLASSIFICATION distilBERT


In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = ['0','1','2','3','4'] 

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.star_rating.values
val_labels = df_valid.star_rating.values
test_labels = df_test.star_rating.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
from transformers import BertForSequenceClassification
from transformers import DistilBertForSequenceClassification

## DEFINIMOS EL MODELO 

model = DistilBertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
device

device(type='cuda', index=0)

## Training

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    #report_to = 'wandb',
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=3,                                 # total number of training epochs
    per_device_train_batch_size=8,                      # batch size per device during training
    per_device_eval_batch_size=16,                      # batch size for evaluation
    warmup_steps=500,                                   # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                  # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',          # directory for storing logs
    logging_steps=10,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
    run_name = 'review_body',     
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,            # training dataset
    eval_dataset = test_data,              # evaluation dataset
    compute_metrics=compute_metrics            
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,1.093900,0.959842,0.578800,0.575157,0.580011,0.578800,41.989600,59.539000
2,0.867000,0.975576,0.573200,0.570219,0.570070,0.573200,41.870500,59.708000


## Evaluation

In [ ]:
trainer.evaluate()

wandb: WARNING Step must only increase in log calls.  Step 37500 < 37501; dropping {'eval/loss': 0.9578923583030701, 'eval/accuracy': 0.5864, 'eval/f1': 0.5806245762934176, 'eval/precision': 0.5808483390543308, 'eval/recall': 0.5864, 'eval/runtime': 89.0555, 'eval/samples_per_second': 28.072, 'train/epoch': 3.0}.


{'epoch': 3.0,
 'eval_accuracy': 0.5864,
 'eval_f1': 0.5806245762934176,
 'eval_loss': 0.9578923583030701,
 'eval_precision': 0.5808483390543308,
 'eval_recall': 0.5864,
 'eval_runtime': 89.0555,
 'eval_samples_per_second': 28.072}

##Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/TFG/Modelos_entrenados/Rating_text/tokenizer_config.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Rating_text/special_tokens_map.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Rating_text/vocab.txt',
 'drive/MyDrive/TFG/Modelos_entrenados/Rating_text/added_tokens.json')

# BertForSequenceClassification - RATING
- Clasificación Rating a partir de review_body + titulo

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = ['0','1','2','3','4'] 
#class_names = ['0','1'] 
#class_names = ['1']

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.star_rating.values
val_labels = df_valid.star_rating.values
test_labels = df_test.star_rating.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
## Lo definimos de nuevo unicamente con el proposito de usar las secciones del documento de forma independiente 

model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
!nvidia-smi

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=3,                                  # total number of training epochs
    per_device_train_batch_size=8,                       # batch size per device during training
    per_device_eval_batch_size=16,                       # batch size for evaluation
    warmup_steps=500,                                    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                   # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
    run_name = 'review_title + review_body', 
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,            # training dataset
    eval_dataset = test_data,              # evaluation dataset
    compute_metrics = compute_metrics             
)

trainer.train()

Epoch,Training Loss,Validation Loss


## Evaluation

In [ ]:
trainer.evaluate()

## Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG/Modelos_entrenados/Rating_text_title"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# BertForSequenceClassification - RATING
- Clasificación Rating a partir de review_body + titulo + categoria

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = ['0','1','2','3','4'] 
#class_names = ['0','1'] 
#class_names = ['1']

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.text_title_cat.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.text_title_cat.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.text_title_cat.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.star_rating.values
val_labels = df_valid.star_rating.values
test_labels = df_test.star_rating.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,                         # total number of training epochs
    per_device_train_batch_size=8,              # batch size per device during training
    per_device_eval_batch_size=16,              # batch size for evaluation
    warmup_steps=500,                           # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                          # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',   # directory for storing logs
    logging_steps=10,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
    run_name = 'review_title + review_body + category', 
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,            # training dataset
    eval_dataset = test_data,              # evaluation dataset
    compute_metrics = compute_metrics       
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,0.787100,0.880430,0.628000,0.624201,0.623436,0.628000,18.008800,55.529000


TrainOutput(global_step=1500, training_loss=1.0608832273483277, metrics={'train_runtime': 732.4994, 'train_samples_per_second': 2.048, 'total_flos': 4036095037440000, 'epoch': 1.0})

## Evaluation

In [ ]:
trainer.evaluate()

{'epoch': 1.0,
 'eval_accuracy': 0.628,
 'eval_f1': 0.6242008828903407,
 'eval_loss': 0.8804295659065247,
 'eval_precision': 0.6234356003347613,
 'eval_recall': 0.628,
 'eval_runtime': 17.8392,
 'eval_samples_per_second': 56.056}

## Save Model

In [ ]:
!ls drive/MyDrive/TFG/Modelos_entrenados/

Rating_text  Rating_text_title	Rating_text_title_cat


In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG/Modelos_entrenados/Rating_text_title_cat"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/TFG/Modelos_entrenados/Rating_text_title_cat/tokenizer_config.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Rating_text_title_cat/special_tokens_map.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Rating_text_title_cat/vocab.txt',
 'drive/MyDrive/TFG/Modelos_entrenados/Rating_text_title_cat/added_tokens.json')

# BertForSequenceClassification - CATEGORÍA

- Clasificación de categorias de productos a partir del review_body

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = df_train.product_category.unique().tolist()

In [ ]:
len(class_names)

31

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.product_category_cod.values
val_labels = df_valid.product_category_cod.values
test_labels = df_test.product_category_cod.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,         # training dataset
    eval_dataset = test_data,
    compute_metrics = compute_metrics            # evaluation dataset
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,2.291400,1.900626,0.484000,0.457461,0.466645,0.484000,17.833100,56.076000


TrainOutput(global_step=1500, training_loss=2.4701984977722167, metrics={'train_runtime': 731.0666, 'train_samples_per_second': 2.052, 'total_flos': 4036832096256000, 'epoch': 1.0})

## Evaluation

In [ ]:
trainer.evaluate()

{'epoch': 1.0,
 'eval_accuracy': 0.484,
 'eval_f1': 0.45746141413361835,
 'eval_loss': 1.9006259441375732,
 'eval_precision': 0.46664481507264,
 'eval_recall': 0.484,
 'eval_runtime': 17.8415,
 'eval_samples_per_second': 56.049}

In [ ]:
preds = trainer.predict(test_data)

In [ ]:
import numpy as np
y_preds = preds.label_ids
print(len(np.unique(y_preds)))
print(np.unique(y_preds))

30
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 24
 25 26 27 28 29 30]


In [ ]:
print(len(np.unique(test_labels)))
print(np.unique(test_labels))

30
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 24
 25 26 27 28 29 30]


In [ ]:
len(class_names)

31

In [ ]:
from sklearn.metrics import classification_report

#print(classification_report(test_labels, y_preds, target_names= class_names

## Save Model

In [ ]:
!ls drive/MyDrive/TFG/Modelos_entrenados/

Category_text	     Rating_text	Rating_text_title_cat
Category_text_title  Rating_text_title


In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG/Modelos_entrenados/Category_text"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/TFG/Modelos_entrenados/Category_text/tokenizer_config.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Category_text/special_tokens_map.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Category_text/vocab.txt',
 'drive/MyDrive/TFG/Modelos_entrenados/Category_text/added_tokens.json')

# BertForSequenceClassification - CATEGORÍA

- Clasificación de categorias de productos a partir del review_body + titulo

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = df_train.product_category.unique().tolist()

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.product_category_cod.values
val_labels = df_valid.product_category_cod.values
test_labels = df_test.product_category_cod.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,         # training dataset
    eval_dataset = test_data,
    compute_metrics = compute_metrics             # evaluation dataset
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,2.349900,1.861099,0.484000,0.465439,0.486365,0.484000,17.859200,55.993000


TrainOutput(global_step=1500, training_loss=2.4554180132548016, metrics={'train_runtime': 731.6949, 'train_samples_per_second': 2.05, 'total_flos': 4036832096256000, 'epoch': 1.0})

## Evaluation

In [ ]:
trainer.evaluate()

{'epoch': 1.0,
 'eval_accuracy': 0.484,
 'eval_f1': 0.4654393747639713,
 'eval_loss': 1.8610986471176147,
 'eval_precision': 0.48636542705370245,
 'eval_recall': 0.484,
 'eval_runtime': 17.8336,
 'eval_samples_per_second': 56.074}

## Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

ls: cannot access 'drive/MyDrive/TFG_1/Modelos_entrenados/': No such file or directory


In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG/Modelos_entrenados/Category_text_title"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/TFG/Modelos_entrenados/Category_text_title/tokenizer_config.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Category_text_title/special_tokens_map.json',
 'drive/MyDrive/TFG/Modelos_entrenados/Category_text_title/vocab.txt',
 'drive/MyDrive/TFG/Modelos_entrenados/Category_text_title/added_tokens.json')